# Домашнее задание 7
**Памперсы или пиво? Практический проект по созданию рекомендательной системы**

**Цель**: Настало время поработать с данными самой большой и самой дорогой компании, занимающейся, похоже, уже всем на свете: от продажи всего и вся, до настройки облачных инфраструктур и создания роботов. Конечно же это компания Джеффа могу-купить-весь-мир Безоса - Amazon.

<hr>

1. Выберите любой понравившийся вам набор данных по ссылке https://nijianmo.github.io/amazon/index.html. Проведите базовый EDA - распределения рейтингов, количество уникальных товаров и т.д. Отложите часть данных для тестирования. В рекомендательных системах для этого можно случайным образом “занулить” желаемый процент рейтингов в исходном датасете, чтобы получить “тренировочный набор”, и проверять качество ваших рекомендаций на этих уже не зануленных рейтингах. При этом, если у вас есть временная зависимость в данных, имеет смысл занулять рейтинги “из будущего”, чтобы обучаться на “исторических” покупках/просмотрах и т.д. 
2. На основании вашего датасета постройте рекомендательную систему. Оцените качество полученных рекомендаций, при помощи подходящих метрик (если вы использовали рейтинги, можно взять RMSE).

<hr>

**Критерии оценки: <li> Базовый EDA - 2 балла. <li> Train-test разбиение - 2 балла. <li> Построение модели и оценка качества - 6 баллов.

# 1. Preprocessing and EDA

Скачаем один из "маленьких"  датасетов, связанных с косметикой. Датасет влючает ID продуктв, ID пользователя, рейтинг и отметка времени. К этому же датасету скачаем метаданные, которые включают текстовое описание продукта. Так будет интереснее смотреть на результаты работы алгоритма.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('All_Beauty.csv', header = None, names = ['item', 'user', 'rating', 'timestamp'])
df.head(4)

,item,user,rating,timestamp
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000


In [3]:
df.describe(include = 'object')

,item,user
count,371345,371345
unique,32586,324038
top,B000FOI48G,A2GJX2KCUSR0EI
freq,8672,27


Итак, датасет содержит довольно много пользователей и продуктов. Такой датасет (как уже проверяла) в полном размере не помещается в память, поэтому немного его уменьшим, оставим только тех пользователей, кто оставил более 2 отзывов.

In [4]:
df_modified = df[df['user'].isin(df['user'].value_counts()[df['user'].value_counts() > 2].index)]
#df_modified = df_modified[df_modified['item'].isin(df_modified['item'].value_counts()[df_modified['item'].value_counts() > 1].index)]

df_modified = df_modified.reset_index()
df_modified.describe(include = 'object')

,item,user
count,23291,23291
unique,5082,6119
top,B0012Y0ZG2,A2GJX2KCUSR0EI
freq,2231,27


In [5]:
import json

desc = {}

with open("meta_All_Beauty.json") as f:
    s = f.read()
    s = '[' + s + ']'
    s = s.replace('\t','')
    s = s.replace('\n',',')
    s = s.replace(',}','}')
    s = s.replace(',]',']')
    data = json.loads(s)
    
    desc = {}
    for record in data:
            id_n = record['asin']
            title = record['title']
            
            desc[id_n] = title
list(desc.items())[0:5]

[('6546546450', "Loud 'N Clear&trade; Personal Sound Amplifier"),
 ('7178680776', 'No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots'),
 ('7250468162', 'No7 Stay Perfect Foundation Cool Vanilla by No7'),
 ('7367905066',
  'Wella Koleston Perfect Hair Colour 44/44 Medium Intense Red Brown 60ml'),
 ('7414204790', 'Lacto Calamine Skin Balance Oil control 120 ml. (Pack of 2)')]

Создадим функцию для разбивки данных на train/test. Разобъем данные как рекомендую в задании, включать в train только последние по времени 20% данных. При этом добавим константу 0,5, запись попадала в train, если значение индекса будет ноль (иначе будет большой дисбаланс и все записи c 1-2 рейтингами уйдут в test). 

In [6]:
from tqdm import tqdm_notebook

def train_test_split(df):
    df.sort_values(by = 'timestamp', inplace = True)
    users_list = df['user'].unique()
    #items_list = df['item'].unique()
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    
    for user in tqdm_notebook(users_list):
        temp = df[df['user'] == user]
        indx = int(temp.shape[0] * (1 - 0.2) + 0.5)
        #print(indx)
        X_train_list.append(temp[['user', 'item']].iloc[:indx, :].values)
        X_test_list.append(temp[['user', 'item']].iloc[indx:, :].values)
        y_train_list.append(temp['rating'].values[:indx])
        y_test_list.append(temp['rating'].values[indx:])
        
    X_train = pd.DataFrame(np.vstack(X_train_list), columns = ['user', 'item'])
    X_test = pd.DataFrame(np.vstack(X_test_list), columns = ['user', 'item'])
    y_train = np.hstack(y_train_list)
    y_test = np.hstack(y_test_list)
    
    return X_train, X_test, y_train, y_test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_modified)
X_train.shape, len(y_train), X_test.shape, len(y_test)

  0%|          | 0/6119 [00:00<?, ?it/s]

((17050, 2), 17050, (6241, 2), 6241)

# 2. Создание рекомендательных систем

## 2.1 Colloborative explicit filtering

Начнем с самой простой и как раз уменьшенные данные пригодятся. При больших кол-вах пользователей и продуктов не зватило памяти для подсчета близости.

Будем использовать user-based model, которая подразумевает поиск схожести пользователей на основе вектора их рейтингов. По оценкам схожих пользователей считается рейтинг для продуктов у искомого пользователя. Для расчета прогнозного рейтинга будем использовать формулу, предложенную на лекции:

$$
    r_{ui} = \overline{r_u} + \frac
    {\sum_{v \in User_i}\big(\textit{sim(u, v)} \times (r_{vi} - \overline{r_v})\big)}
    {\sum_{v \in User_i}\textit{|sim(u, v)|}}
$$

#### Шаг 1. 
Посчитаем средний рейтинг по каждому пользователю и вычетм его из всех его рейтингов.

In [8]:
X = X_train.copy()
users = X['user'].unique()
items = X['item'].unique()
X['y'] = y_train
mean_user = X.groupby('user')['y'].mean()
X['y'] -= X['user'].apply(lambda x: mean_user[x])

#### Шаг 2. 
Создадим матрицу user/item, кол-во строк которой - кол-во уникальных пользователей, кол-во стобцов - кол-во уникальных продуктов. Получиться большая разреженная матрица.
Для этой же матрицы посчитаем схожесть каждого пользователя с каждым по вектору их рейтингов. В результате будет массив, где кол-во строк и стобцов равно кол-ву уникальных пользователей.

In [9]:
user_vectors = pd.pivot_table(X, values = 'y', index = 'user', columns = 'item',
                             fill_value = 0)
print(f'User/item matrix shape: {user_vectors.shape}')
user_vectors.head(2)

User/item matrix shape: (6119, 3960)


item,0992916305,1465042776,1465045953,1620213982,4293845755,9742121109,9790787006,B000050B63,B000050B65,B000050B6B,...,B01HBWYB5Y,B01HBXID8Y,B01HCPNYR6,B01HD23OJG,B01HE4QV52,B01HFLLNXE,B01HG2R3KE,B01HHWBYNK,B01HIPOQ2M,B01HIWLLUK
user,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0,0,0,0.0,0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
A100WO06OQR8BQ,0,0,0,0.0,0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(user_vectors)
print(f'User similarity matrix shape: {user_sim.shape}')
user_sim[0:2]

User similarity matrix shape: (6119, 6119)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Шаг 3. 
Подсчитаем позицию каждого пользователя в матрице user/item - это нужно для формулы предсказания рейтинга. Постороим функцию для прогноза рейтинга. Предскажим рейтинг для тестового набора и посмотрим не метрики для оценки качества регрессии.

In [11]:
user_pos = dict()
for user in users:
    user_pos[user] = np.argwhere(user_vectors.index.values == user)[0][0]
    
def predict_rating(user, item):

    if not item in items or not user in users:
        return 0

    
    numerator = user_sim[user_pos[user]].dot(
                    user_vectors.loc[:, item])
    
    denominator = round(np.abs(user_sim[user_pos[user]]).sum() - 1, 4) 
    # округлим до 4-х знаков, а то очень малые числа в знаменателе могут дать очень большие числа для к-та
    
    if denominator == 0:
        return 0

    return mean_user[user] + numerator / denominator

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

y_pred = X_test[['user', 'item']].apply(lambda row: predict_rating(row[0], row[1]), axis = 1)

print(f'RMSE Colloborative explicit filtering {round(np.sqrt(mean_squared_error(y_test, y_pred)),4)}')
print(f'MAE Colloborative explicit filtering {round(mean_absolute_error(y_test, y_pred),4)}')
print(f'MAPE Colloborative explicit filtering {round(mean_absolute_percentage_error(y_test, y_pred),4)}')
print(f'R2 Colloborative explicit filtering {round(r2_score(y_test, y_pred),4)}')

RMSE Colloborative explicit filtering 2.1926
MAE Colloborative explicit filtering 1.2391
MAPE Colloborative explicit filtering 0.3513
R2 Colloborative explicit filtering -2.8438


Результаты конечно не очень хорошие, в среднем не тестовом датасете алгоритм ошибается на 2 пункта. Посмотрим, как выглядят точечные рекомедации для пользователей. Будем брать только тех, которые были в трейн сете, так как алгоритм не знает новых пользователей.

Будем перебирать все товары в трейн-сете для конткретного пользователя, затем из отберем топ-рекомендации - все больше 3.

Для новых пользователей это может быть отдельное правило, например, топ-5 по рекомендациям в категории.

In [13]:
train_user_list = X_train['user'].unique()
train_item_list = X_train['item'].unique()

def make_prediction(user):
    ratings_pred = {}

    for i in train_item_list:
        rating = predict_rating(user, i)
        if rating > 3:
            ratings_pred[i] = rating

    purchase_list = df_modified[df_modified['user'] == user]['item'].values

    print('Purchased:')
    for item in purchase_list:
        try:
            print(f'***{desc[item][:100]}')
        except:
            print(item)
            
    print('\n')
    print(f'Recomendations:')
    
    sorted_recs = []
    for i in sorted(ratings_pred, key=ratings_pred.get, reverse=True):
        if i in purchase_list:
            pass
        else:
            sorted_recs.append(i)
            
    for recom in sorted_recs[:5]:
       try:
            print(f'***{desc[recom][:100]}')
       except:
            print(recom)
            
    return list(ratings_pred.items())[0:5]

In [14]:
import random
make_prediction(random.choice(train_user_list))

Purchased:
***Gillette Venus &amp; Olay Sugarberry Scent Women's Razor Blade Refills 3 Count
***NARS Blush, Gaiety
***NARS Blush, Taj Mahal


Recomendations:
***Norelco 6885XL Deluxe Quadra Action Cord/Cordless Rechargeable Men's Shaver
***Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging May Vary)
***Braun 8000 Activator Combi-Pack Foil and Cutterblock Replacement Parts for Braun's Activator Razor M
***Norelco 6826XL Quadra Action Cord/Cordless Rechargeable Men's Shaver
***Method Hand Wash, Pomegranate - 12 fl oz


[('B000050B65', 5.0),
 ('B000050FDY', 5.0),
 ('B0002MQ9GK', 5.0),
 ('B000050B63', 5.0),
 ('B00027D5H6', 5.0)]

In [15]:
make_prediction(random.choice(train_user_list))

Purchased:
***Fresh Guard Wipes Specially Formulated for Retainers Mouthguards and Removable Braces, 20 Count
***dr. brandt Cellusculpt, 6.3 fl. oz.
***Head and Shoulders Green Apple 2-In-1 Dandruff Shampoo And Conditioner 23.7 F
***MHU Professional Folding Blow Dryer 1875w Negative Ion Hair Dryer Dual Voltage Dc Motor Lightweight 
***Star Tech Professional Grade Lady Beauty Tool Eyelash Curler (Pink)
***Neutrogena Healthy Defense Daily Moisturizer Sensitive Skin, SPF 50 Lotion 1.70 oz


Recomendations:
***essie nail polish, cuticle care, primers and finishers
***essie Gel Couture Nail Polish
***Pantene Pro-V Volume Conditioner 12.0 Fluid Ounce (Product Size May Vary)
***Gillette M3 Power Cartridges 8 Count
***Toni&amp;Guy Glamour Volume Plumping Whip, 2.82 Fluid Ounce


[('B000050B65', 4.8),
 ('B000050FDY', 4.8),
 ('B0002MQ9GK', 4.8),
 ('B000050B63', 4.8),
 ('B00027D5H6', 4.8)]

In [16]:
make_prediction(random.choice(train_user_list))

Purchased:
***Glitter Eyeshadow Diamond Dust Professional Grade 48 Color Day &amp; Night Eyeshadow 2 Palette Set
***FOONEE Rhinestones Nail Art Gems Mixed Colours Shapes in Case (2mm,3000pcs)
***Freeman Bare Foot Exfoliating foot scrub Peppermint and Plum 5.3 oz (Packs of 3)
***Next Image/ON Organic Natural Hair Care Products Gift Set


Recomendations:
***Chialstar Lip Gloss Stick Makeup Waterproof Velvet Matte 24 Full Color Lipstick and Chialstar Storag
***Italia Deluxe Ultra Fine Lip Liner set (Pack Of 12)
***Liquid Lip Gloss Makeup With Applicator Wand In Long Lasting Glossy Gorgeous Colors - Lipstick &amp;
***Sinful Colors Professional Nail Polish Enamel 56 Neon Melon
***essie Gel Couture Nail Polish


[('B000050B65', 4.333333333333333),
 ('B000050FDY', 4.333333333333333),
 ('B0002MQ9GK', 4.333333333333333),
 ('B000050B63', 4.333333333333333),
 ('B00027D5H6', 4.333333333333333)]

Вместе с рекомендацией вывела перечень рейтингов, большинтсву продуктов модель одинаковые рейтинги, но при этом смогла подобрать какие-то рекомендации. Порой странные, но вроде бы разные для всех тестовых пользователей. При предыдущем прогоне ноутбука у меня были для всех 3-х слечайно отобранных пользователей все одинаковые рекомендации.


Посмотри, как будет работать готовая библиотека.


## 2.2 Surprise

Импортируем библиотеку и подготовим датасеты в том формате, которой для нее нужен.

In [17]:
import surprise

def data_preprocess(X,y):
    X = X.copy()
    X['rating'] = y
    reader = surprise.Reader(line_format='user item rating', rating_scale=(1, 5))
    dataset = surprise.Dataset.load_from_df(X[['user', 'item', 'rating']], reader)
    return dataset

train_dataset = data_preprocess(X_train,y_train)
test_dataset = data_preprocess(X_test,y_test)

Создадим функцию, которая будет выводить результаты работы разных моделей из библиотеки surprise на нашем тестовом датасете и запустим ее на основных моделях с некими общими параметрами.

In [18]:
def list_predictions(model, trainset,testset):
    train = trainset.build_full_trainset()
    model.fit(train)
    test = [testset.df.loc[i].to_list() for i in range(len(testset.df))]
    y_pred = []
    for prediction in model.test(test):
        y_pred.append(prediction[3])
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    MAE = mean_absolute_error(y_test, y_pred)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    R2 = r2_score(y_test, y_pred)
    return RMSE, MAE, MAPE, R2

In [19]:
models = [surprise.BaselineOnly(),
          surprise.SVD(n_factors=20, n_epochs=30, lr_all=0.005, reg_all=0.02, random_state=42),
          surprise.SVDpp(n_factors=20, n_epochs=30, lr_all=0.005, reg_all=0.02, random_state=42),
          surprise.SlopeOne(),
          surprise.CoClustering(n_cltr_u = 6, n_cltr_i = 6, n_epochs=30, random_state=42),
          surprise.KNNBaseline(k = 20, min_k = 2, user_based = True)
         ]

names = ['Baseline', 'SVD','SVD+', 'SlopeOne', 'CoClust', 'KNN_base']

In [20]:
%%time

scores = pd.DataFrame()
for name, model in zip(names, models):
    print(f'Working over model {name}...')
    RMSE, MAE, MAPE, R2 = list_predictions(model, train_dataset,test_dataset)
    new_row = {'Test':name, 
               'RMSE':round(RMSE,4),
               'MAE':round(MAE,4),
               'MAPE': round(MAPE,4),
               'R2_score': round(R2,4)
               }
    scores = scores.append(new_row, ignore_index=True)
scores = scores.set_index('Test')
display(scores.sort_values(by = 'RMSE'))

Working over model Baseline...
Estimating biases using als...
Working over model SVD...
Working over model SVD+...
Working over model SlopeOne...
Working over model CoClust...
Working over model KNN_base...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


,RMSE,MAE,MAPE,R2_score
Test,,,,
KNN_base,0.9303,0.5379,0.2515,0.3080
SVD,0.9566,0.6361,0.2817,0.2684
SVD+,0.9709,0.6803,0.2878,0.2464
SlopeOne,1.0073,0.5411,0.2363,0.1887
CoClust,1.0394,0.5956,0.2513,0.1362
Baseline,1.0428,0.7120,0.3217,0.1306


Wall time: 3.05 s


Для первых двух лучших алгоритмов попробуем подобрать параметры с помощью GridSearch. Интересно, что SVD+ только на 3-м месте.

In [21]:
%%time

param_grid = {'n_factors': [6, 10, 20, 40, 100], 'n_epochs': [50,], 
               'lr_all': [0.005, 0.001], 'reg_all': [0.05, 0.02],
              'random_state': [42, ]
             }

gs_svd = surprise.model_selection.GridSearchCV(surprise.SVD, param_grid,
                                               measures=["rmse"], cv=5)

gs_svd.fit(train_dataset)

print(f'{gs_svd.best_score["rmse"]}\n')
print(f'{vars(gs_svd.best_estimator["rmse"])}')

0.8578258486849311

{'n_factors': 100, 'n_epochs': 50, 'biased': True, 'init_mean': 0, 'init_std_dev': 0.1, 'lr_bu': 0.005, 'lr_bi': 0.005, 'lr_pu': 0.005, 'lr_qi': 0.005, 'reg_bu': 0.02, 'reg_bi': 0.02, 'reg_pu': 0.02, 'reg_qi': 0.02, 'random_state': 42, 'verbose': False, 'bsl_options': {}, 'sim_options': {'user_based': True}}
Wall time: 8.45 s


In [22]:
%%time

param_grid = {'k': [5, 10, 20, 40], 'min_k': [2,], 
              'sim_options': {
                  'name': ['cosine'],
                  'user_based': [True, False]
                  }
             }

gs_knn = surprise.model_selection.GridSearchCV(surprise.KNNBaseline, param_grid, 
                                           measures=["rmse"], cv=5)

gs_knn.fit(train_dataset)

print(f'{gs_knn.best_score["rmse"]}\n')
print(f'{vars(gs_knn.best_estimator["rmse"])}')

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Comput

SVD при подборе параметров показал результат лучше. Посмотрим, что он покажет на кросс-валидации и на тестовом датасете.

In [23]:
algo_best = surprise.SVD(n_factors = 100, n_epochs = 50, 
              lr_all = 0.005, reg_all = 0.02,
              random_state = 42)
surprise.model_selection.cross_validate(algo_best, train_dataset, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8883  0.8526  0.8331  0.8234  0.9019  0.8599  0.0306  
MAE (testset)     0.5432  0.5301  0.5131  0.5051  0.5496  0.5282  0.0170  
Fit time          0.14    0.14    0.14    0.15    0.14    0.14    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.88829312, 0.85264969, 0.8331153 , 0.82335011, 0.90190201]),
 'test_mae': array([0.54320292, 0.53007491, 0.51309192, 0.50507291, 0.54961298]),
 'fit_time': (0.13899993896484375,
  0.1379995346069336,
  0.14002323150634766,
  0.14989638328552246,
  0.1379997730255127),
 'test_time': (0.00699925422668457,
  0.006999969482421875,
  0.007999420166015625,
  0.007999420166015625,
  0.007999658584594727)}

In [24]:
RMSE, MAE, MAPE, R2 = list_predictions(algo_best, train_dataset,test_dataset)

print(f'RMSE: {round(RMSE,4)}, MAE: {round(MAE,4)}, MAPE: {round(MAPE,4)}, R2: {round(R2,4)}')

RMSE: 0.9165, MAE: 0.5595, MAPE: 0.2527, R2: 0.3284


Модель получилась немного переобученной, так как RMSE выше на тесте, чем на трейне. В целом результат в 2 раза лучше, чем в случае с коллоборативной фильтрацией.

Следующий шаг - посмотрим, как она дает точечные предсказания для отдельных пользователей. Для этого воспользуемся функцией build_anti_testset в библиотеке. Эту функцию будем исползовать для всего набора данных. Она вернет сочетания пользователей, продуктов и рейтингов, которых нет в датасете. Посмотрим, какие рекомендации даст алгоритм для таких пользователей. Будем выводить список их покупок и рекомендаций.

In [25]:
total_dataset = data_preprocess(df_modified[['user', 'item']],df_modified['rating'])
trainset = total_dataset.build_full_trainset()
testset = trainset.build_anti_testset()
len(testset)

31076676

In [26]:
testset[0:10]

[('A141OPVE376YFI', 'B000050B63', 4.447254304237688),
 ('A141OPVE376YFI', 'B000068PBJ', 4.447254304237688),
 ('A141OPVE376YFI', 'B00005U8U8', 4.447254304237688),
 ('A141OPVE376YFI', 'B000068PBO', 4.447254304237688),
 ('A141OPVE376YFI', 'B000065AB1', 4.447254304237688),
 ('A141OPVE376YFI', 'B00005355V', 4.447254304237688),
 ('A141OPVE376YFI', 'B000068PBK', 4.447254304237688),
 ('A141OPVE376YFI', 'B00005354G', 4.447254304237688),
 ('A141OPVE376YFI', 'B000089SAQ', 4.447254304237688),
 ('A141OPVE376YFI', 'B00009RB14', 4.447254304237688)]

In [27]:
from collections import defaultdict

def user_prediction(user):
    purchase_list = df_modified[df_modified['user'] == user]['item'].values
    print('Purchased:')
    for item in purchase_list:
        try:
            print(f'***{desc[item][:100]}')
        except:
            print(item)
            
    testset_1 = (el for el in testset if el[0] == user)
    predictions = algo_best.test(testset_1)
    
    top = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top[uid].append((iid, est))
    for uid, user_ratings in top.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top[uid] = user_ratings
        
    print('\n')
    print(f'Recomendations:')

    recomendations = [p[0] for p in top[user]][:5]

    for recom in recomendations:
           try:
                print(f'***{desc[recom][:100]}')
           except:
                print(recom)

In [28]:
total_users = df_modified['user'].unique()

user_prediction(random.choice(total_users))

Purchased:
***Bonne Bell Smackers Bath and Body Starburst Collection
***Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
***Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
***Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.
***Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz


Recomendations:
***Truefitt &amp; Hill Trafalgar After Shave Splash 100ml/3.38oz
***Commonwealth Lavare Extra Large Pineapple Bath Soap 12 Oz. by CST
***Paul Brown Hawaii Hapuna Hair Styling Paste, 8 Ounce
***Boots No7 Moisture Quench Day Fluid 3.3 fl oz (100 ml)
***Gillette All Over Clean Hair &amp; Body Wash 16 Fl Oz (Pack of 2)


In [29]:
user_prediction(random.choice(total_users))

Purchased:
***Citre Shine Moisture Burst Shampoo - 16 fl oz
***Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
***Bonne Bell Smackers Bath and Body Starburst Collection
***Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
***Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz


Recomendations:
***Rainbow Light - MintAsure Fresh Breath Capsules, Powerful Support for Healthy Teeth and Gums and Las
***Neutrogena Ultra Sheer Dry-Touch Sunscreen Broad Spectrum SPF 70, 3 Fl. oz.
*** Supernail China Silk Wrap, 72 Inch 
***EVO FLEX Sunnies Flexible Tanning Bed Goggles Eye Protection UV PINK Glasses
***Pack of 6 Salon Sectioning Clip Clamp Hairdressing Styling Hair Tool Crocodile


In [30]:
user_prediction(random.choice(total_users))

Purchased:
***Panasonic WES9839P Men's Electric Razor Replacement Inner Blade &amp; Outer Foil Set
***Crest Pro-Health Clinical Deep Clean Mint Mouthwash 473 Ml
***Dr. Bronner's Organic Castile Bar Soap - Almond - 5 oz.


Recomendations:
***Norelco 6826XL Quadra Action Cord/Cordless Rechargeable Men's Shaver
***Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.
***Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
***bareMinerals Essential Brow Kit - Dark Blond/Medium Brown
***Alpha Hydrox AHA Souffle Soothing Anti-Wrinkle 1.6 oz.


Итак, здесь предсказания для индивидуального пользователя уже намного лучше и можно сказать ей некий смысл в них (в последнем случае предлагают набор для окрашивания бровей пользователю, купившему съемные лезвия для бритв).Теперь я знаю, как появляются раздражащие рекомендации.

Однако, как нам говорили на занятии, качество рекомендательной системы оценивается бизнес-метриками. Кроме того, на практике не используется один алгоритм, а несколько.